In [5]:
import numpy as np
import os 
import sys

import Alpaga
from Alpaga import file_management
from Alpaga.Data_tutorial import get_tutorial_path 


# Introduction (please read)

In this tutorial, we will see the different functions available to read data obtained from our experimental device. he **main idea** is that one obtain experimentally a serie of **iteration** as a function of a "parameters", that we will call **angle** -- historically, this is the light polarization. The goal of this tutorial is to explain you how to connect these file to Alpaga for their analyses. 

The iteration are a serie of acquisition that should sample the same spectra. We want to average the spectra over the iteration for a given angle. 
Pratically speaking, if you are recording one spectra over 10 sec for each angle, you will have 1 iterations per angle. If you are recording 10 spectra over 1 sec for each angle, you will have 10 iterations per angle. 

If you are recording spectra as a function of angle, for instance let us say from 0 to 180 with 10 step, you will have 19 angles. If you have 10 iteration per angle, you will get 19x10=190 spectra to treat. 

Again, we would like to stress that angle can represent something else then angle. Technically speacking, they are used as strings in Alpaga. 

We will in the spectra_analysis tutorial how to do it with Alpaga, but first, let us discuss how to access the data... **your** data. In fact, this is probably the part where you will have to spend some time during your first Alapaga.

Let us first present how it is done initially at the lab where Alpaga was created. 
The LabView controlling the acquisition saves the spectra in a directory as: 

prefix + _ + angle value + _ + iteration number  + .dat 

For instance, for an angle of 0.0 and with 3 acquisitions, we would have 3 files: polarP_0.0_1.dat, polarP_0.0_2.dat and polarP_0.0_3.dat. 


Given a prefix, angle_value and iteration, the function *alpaga.file_management.standard_file_name* built the file name using the presented rule:


In [6]:
name_out = Alpaga.file_management.standard_file_name('prefixe', angle='42.0', iteration='4', extention='.dat')
print(name_out)

name_out = Alpaga.file_management.standard_file_name('prefixe', angle=False, iteration='4', extention='.dat')
print(name_out)

name_out = Alpaga.file_management.standard_file_name('prefixe', angle='42.0', iteration=False, extention='.dat')
print(name_out)

name_out = Alpaga.file_management.standard_file_name('prefixe', angle=False, iteration=False, extention='.dat')
print(name_out)

prefixe_42.0_4.dat
prefixe_4.dat
prefixe_42.0.dat
prefixe.dat


The whole Alpaga analysis is built around this notation of **angle**, for the "physical" analyses, and **iteration** , for denoising and computing the experimental errors.

Hence, either you are using this rule for naming your files, or very similar one, or not. 
  + If you are, follow the next part and you will probably have few difficulties to use directly the next tutorials for your analyses. 
  + If no, go to the last part. 
  
In any case, Alpaga will need a way to access the files and to relate them to a given **angle** and **iteration**. 

**Note**: If you are saving your iteration for a single iteration one after another, please contact us, this case is not yet handle -- but should be easely fixed. 

# About the Data for the tutorials

The data for the tutorials are within Alpaga package. You can access them using the *Alpaga.Data_tutorial.get_tutorial_path* function. This is already done in the tutorials so you should not need to worry about this part. 

If you are looking for more data / open the data by yourself, have a look at Alpaga/Data_tutorial in the github/your version of the repository, or look in the directory returned by: 

In [7]:
print('Where is store the tutorial data in your case:', get_tutorial_path(""))

Where is store the tutorial data in your case: /home/glebreton/.local/lib/python3.7/site-packages/Alpaga/Data_tutorial/


# "Usual" naming convention

If you are reading this part, your naming convention respects something like:

prefix + _ + angle value + _ + iteration number  + extension

and your files are saved **IN THE SAME DIRECTORY** for a given experiment. 
 
If your convention is sightly different, read this part and the next one.


If you have recorded only one type of experiment (**i.e.** one prefix), you can use either the function *find_file_iter_from_dir* or *alpaga.find_angle_iter_from_dir* to find automatically the number of iterations and angles.

The following example are self-explanatory:

In [8]:
directory = get_tutorial_path("SHS/Eau_polar_V")
prefix_file, N_iter, extention = Alpaga.file_management.find_file_iter_from_dir(directory)
print('The prefix for all the file are: "' + prefix_file + '" with ' + str(N_iter) + ' iter. The extention is: ' + extention)

I will look at file with the extention .dat in the directory /home/glebreton/.local/lib/python3.7/site-packages/Alpaga/Data_tutorial/SHS/Eau_polar_V for single acquisition. The type of the files should be: prefix_iter.extention


Exception: There are other file with the same extention but not the same prefix name. Please move these file, or do not use this module to creat the list! If you are looking for angle depend value, use the find_angle_iter_from_dir function.

In [ ]:
directory = os.path.join(Dir_tuto_file, 'SHS/Eau_polar_V')
prefix_file, L_files_angles, N_iter, extention = Alpaga.file_management.find_angle_iter_from_dir(directory)
print('The prefix for all the file are: "' + prefix_file + '" with ' + str(N_iter) + ' iter. The angle found are ' + str(L_files_angles) + '. The extention is: ' + extention)

Using these function, one can obtained directly the list of "angles" and "iteration" if the naming convention is the same as ours. 

If it is not the case, you can still look at these functions to build yours. These functions are not mendatory for Alpaga uses. 

# "Very close" naming convention

Let us say that your are using a slightly different naming convention, for instance: prefix-$anglevalue-$iteration.extension, or prefixe_angleval$anglevalue_aq$iteration.dat. 

Then, you will not be able to directly use the function defined in Alpaga.file_management and you will need to provide an equivalent. 

If you want to say close to the initial Alpaga way, you have to give a "convention" and a "finders" functions to replace *Alpaga.file_management.standard_file_name* and *Alpaga.file_management.find_angle_iter_from_dir*, respectively. 

For instance for the "convention" function:

In [ ]:
def my_own_convention_for_filename(prefixe, angle=False, iteration=False, extention='.dat'):
    '''
    Define how to built the name of a file. 
    Please note that the function structure / argument / output should be exactly like that!!!!
    '''
    if isinstance(angle, bool): # case where no angle is given 
        if isinstance(iteration, bool): # case where no iteration is given 
            name = prefixe + extention
        else: # case where an iteration is given 
            name = prefixe + '_aq' + iteration + extention
    else: # case where an angle is given 
        if isinstance(iteration, bool):  # case where no iteration is given 
            name = prefixe + '_' + angle + extention
        else:  # case where an iteration is given 
            name = prefixe + '_' + angle + '_aq' + iteration + extention
    return(name)
    
name_out = my_own_convention_for_filename('prefixe', angle='42.0', iteration='4', extention='.dat')
print(name_out)

We recommand to built a function that gives you a list of angles and a number of iteration per acquisition automatically just by knowing the directory name, but it is not mendatory. 
If you do not built this function, you will have to provide:
  + prefix 
  + the iteration number, N_iter
  + the angle list, L_angle  
  + extension
  + A way to link a prefix + angle value + iteration number to a filename. 
  
Note: the prefix can be an empty string if required.

If you are using a very close naming convention, we recommand to define your own *my_own_convention_for_filename*, and you will be able to follow the next tutorial (tutorial_spectra_analysis) 

In [ ]:
# Try out:
def my_own_convention_for_filename(prefixe, angle=False, iteration=False, extention='.dat'):
    '''
    Define how to built the name of a file. 
    Please note that the function structure / argument / output should be exactly like that!!!!
    '''
    if isinstance(angle, bool):  
        if isinstance(iteration, bool):  
            name = prefixe + extention # CHANGE THIS PART 
        else: 
            name = prefixe + '_aq' + iteration + extention
    else: 
        if isinstance(iteration, bool):  # CHANGE THIS PART 
            name = prefixe + '_' + angle + extention
        else:  
            name = prefixe + '_' + angle + '_aq' + iteration + extention # CHANGE THIS PART 
    return(name)
    
name_out = my_own_convention_for_filename('prefixe', angle='42.0', iteration='4', extention='.dat')
print(name_out)

# Different naming convention

If you are using a whole different way of naming / saving the spectra, you should provide the full path of each acquisition as a N_angle x N_iter list. For instance, for 2 angles and 3 iterations:
```python
  L_filename = [ 
  [filename_angle1_iter1, filename_angle1_iter2, filename_angle1_iter3]
  [filename_angle2_iter1, filename_angle2_iter2, filename_angle2_iter3]
  ]
```  

With the good N_iter and any list for the angle:

```python, 
  L_files_angles = ['angle_value_1', 'angle_value_2']
  N_iter = 2 
```  

If you are doing then take care in the following tutorial during the denoising part (part I) and for the file argument for the core function *Alpaga.analyze_run.polarisation_intensity* (part III).